In [1]:
%load_ext autoreload
%autoreload 2

import sys
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("../../..")

import lib.ehr.example_datasets.mimiciv_aki as m4aki
from lib.ehr.tvx_ehr import TVxEHR
from lib.utils import modified_environ, write_config


In [2]:
# create a database connection
sqluser = 'postgres'
dbname = 'mimiciv'
hostname = 'localhost'
password = 'qwerasdf'
port = '5432'

url = f'postgresql+psycopg2://{sqluser}:{password}@{hostname}:{port}/{dbname}'

with modified_environ(MIMICIV_URL=url):
    mimiciv_config = m4aki.AKIMIMICIVDatasetConfig()
    tvx_config = m4aki.TVxAKIMIMICIVDatasetConfig()

    write_config(mimiciv_config.to_dict(), 'dataset_mimiciv_aki_config.json')
    write_config(tvx_config.to_dict(), 'tvx_mimiciv_aki_config.json')


In [3]:
# with modified_environ(MIMICIV_URL=url):
#     dataset = m4aki.AKIMIMICIVDataset(config=mimiciv_config)           
# dataset = dataset.execute_pipeline()
# tvx = m4aki.TVxAKIMIMICIVDataset(config=tvx_config, dataset=dataset)
# tvx = tvx.execute_pipeline()

In [4]:
# tvx.dataset.pipeline_report.to_csv('dataset_pipeline_report.csv')
# tvx.pipeline_report.to_csv('tvx_pipeline_report.csv')
# tvx.save('tvx_aki.h5', True)

In [5]:
# tvx = m4aki.TVxAKIMIMICIVDataset.load('tvx_aki.h5')

In [6]:
# from lib.ehr.tvx_transformations import TrainingSplitGroups
# tvx_list = TrainingSplitGroups()(tvx, n_groups=10, seed=0)

In [7]:
# for i, tvx_item in enumerate(tvx_list):
#     tvx_item.save(f'tvx_aki_training_groups/tvx_aki_{i}.h5', True)

In [8]:
tvx0 = m4aki.TVxAKIMIMICIVDataset.load('tvx_aki_training_groups/tvx_aki_0.h5')

In [9]:
len(tvx0.subjects)

3006

In [10]:
from lib.ml.embeddings import InICENODEEmbeddingsConfig, InterventionsEmbeddingsConfig
from lib.ml.in_models import InpatientModelConfig, InICENODELite
from lib.ml.model import Precomputes

In [11]:
emb_config = InICENODEEmbeddingsConfig(dx_codes=50, demographic=10, 
                                       interventions=InterventionsEmbeddingsConfig(icu_inputs=10,
                                                                                   icu_procedures=10,
                                                                                   hosp_procedures=10,
                                                                                   interventions=20))

model_config = InpatientModelConfig(state=50, lead_predictor='monotonic')

In [12]:
import jax.random as jrandom

model = InICENODELite.from_tvx_ehr(tvx_ehr=tvx0, config=model_config, embeddings_config=emb_config)
                  

2024-05-08 16:26:10.010978: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [13]:
tvx0.config.leading_observable

LeadingObservableExtractorConfig(
  observable_code='renal_aki.aki_binary',
  scheme='mimiciv.aki_study.obs',
  leading_hours=[6.0, 12.0, 24.0, 48.0, 72.0],
  entry_neglect_window=6.0,
  minimum_acquisitions=2,
  recovery_window=12.0
)

In [14]:
tvx0.subjects['10002760'].admissions[0]

SegmentedAdmission(
  admission_id='28094813',
  admission_dates=AdmissionDates(
    admission=Timestamp('2141-04-20 07:15:00'),
    discharge=Timestamp('2141-04-24 13:31:00')
  ),
  dx_codes=CodesVector(vec=bool[17375](numpy), scheme='dx_icd9'),
  dx_codes_history=CodesVector(vec=bool[17375](numpy), scheme='dx_icd9'),
  outcome=CodesVector(vec=bool[11535](numpy), scheme='dx_icd9_v1'),
  observables=SegmentedInpatientObservables(
    time=f64[62](numpy),
    value=f16[62,100](numpy),
    mask=bool[62,100](numpy),
    extra_layers=(),
    indexed_split=i64[61](numpy)
  ),
  interventions=SegmentedInpatientInterventions(
    time=f64[100](numpy),
    icu_inputs=f16[99,320](numpy),
    icu_procedures=bool[99,23](numpy),
    hosp_procedures=None
  ),
  leading_observable=SegmentedInpatientObservables(
    time=f64[62](numpy),
    value=f64[62,5](numpy),
    mask=bool[62,5](numpy),
    extra_layers=(),
    indexed_split=i64[61](numpy)
  )
)

In [15]:
adm = tvx0.subjects['10002760'].admissions[0]
admission_emb = model.f_emb(adm, tvx0.admission_demographics[adm.admission_id])

In [16]:
adm.leading_observable.mask

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  Tr

In [17]:
out = model(admission=adm, embedded_admission=admission_emb, precomputes=Precomputes())

In [18]:
out.leading_observable.value

Array([[0.0966782 , 0.19454093, 0.29660878, 0.36838916, 0.4557795 ],
       [0.09867333, 0.19896655, 0.3045075 , 0.37886837, 0.467533  ],
       [0.09875115, 0.19728632, 0.29887462, 0.37391573, 0.46282706],
       [0.09927674, 0.19696286, 0.2973865 , 0.37288442, 0.4618947 ],
       [0.09974988, 0.19704977, 0.29715863, 0.37304276, 0.4622225 ],
       [0.10000454, 0.19732764, 0.30007407, 0.3784217 , 0.4674242 ],
       [0.09902725, 0.19384083, 0.2940572 , 0.36961794, 0.4580361 ],
       [0.09760293, 0.19187033, 0.29134798, 0.36399955, 0.45201167],
       [0.0997486 , 0.19841696, 0.30124795, 0.37734216, 0.4663714 ],
       [0.09791612, 0.198703  , 0.30414695, 0.38060912, 0.4684422 ],
       [0.09683371, 0.19850868, 0.30482972, 0.38123897, 0.46859574],
       [0.09798314, 0.20044656, 0.3057827 , 0.3830766 , 0.4717048 ],
       [0.09731258, 0.1956862 , 0.29815924, 0.37664837, 0.46493492],
       [0.09763475, 0.19839062, 0.30324957, 0.38137835, 0.47074044],
       [0.09687101, 0.19537607, 0.

In [19]:
from lib.ml.experiment import Experiment, ExperimentConfig
from lib.ml.trainer import Trainer, TrainerConfig, OptimizerConfig, ReportingConfig, LossMixer

opt = OptimizerConfig()
trainer_config=TrainerConfig(obs_loss='mse', lead_loss='mse', optimizer=opt)
reporting_config = ReportingConfig(output_dir='test',
                                   console=True,
                                   parameter_snapshots=True,
                                   config_json=True,
                                   model_stats=False)
loss_mix = LossMixer()


In [22]:
out

AdmissionTrajectoryPrediction(
  admission=SegmentedAdmission(
    admission_id='28094813',
    admission_dates=AdmissionDates(
      admission=Timestamp('2141-04-20 07:15:00'),
      discharge=Timestamp('2141-04-24 13:31:00')
    ),
    dx_codes=CodesVector(vec=bool[17375](numpy), scheme='dx_icd9'),
    dx_codes_history=CodesVector(vec=bool[17375](numpy), scheme='dx_icd9'),
    outcome=CodesVector(vec=bool[11535](numpy), scheme='dx_icd9_v1'),
    observables=SegmentedInpatientObservables(
      time=f64[62](numpy),
      value=f16[62,100](numpy),
      mask=bool[62,100](numpy),
      extra_layers=(),
      indexed_split=i64[61](numpy)
    ),
    interventions=SegmentedInpatientInterventions(
      time=f64[100](numpy),
      icu_inputs=f16[99,320](numpy),
      icu_procedures=bool[99,23](numpy),
      hosp_procedures=None
    ),
    leading_observable=SegmentedInpatientObservables(
      time=f64[62](numpy),
      value=f64[62,5](numpy),
      mask=bool[62,5](numpy),
      extra_layer

In [20]:
experiment_config = ExperimentConfig(model=model_config,
                                       embeddings=emb_config,
                                       trainer=trainer_config,
                                       model_classname='InICENODE',
                                       trainer_classname='Trainer',
                                       reporting=reporting_config,
                                       model_snapshot_frequency=10,
                                     continue_training=True,
                                     loss_mixer=loss_mix)
                                     
               
               

In [21]:
from lib.utils import write_config
write_config(experiment_config.to_dict(), 'config_template.json')

In [6]:
import equinox as eqx
import jax.numpy as jnp

eqx.filter_vmap(lambda a, b,c: a+b+c)(jnp.arange(10), jnp.arange(10), None)

TypeError: unsupported operand type(s) for +: 'BatchTracer' and 'NoneType'